In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel("Export_reference__mots_clefs.xlsx")
df

In [ ]:
df.dtypes

In [ ]:
df = df.drop(["Unnamed: 3","Unnamed: 4"], axis=1).astype("str")


In [ ]:
df_mot_cle = df["MOTCLE"]

def split_word(el : str):
    return el.split(";")
df_mot_cle = df_mot_cle.apply(split_word)
df_mot_cle

In [ ]:
import random
random.shuffle(df_mot_cle)
df_mot_cle[:100]

Perform a One hot encoding with sklearn

In [ ]:
def delete_quote(l: [str]):
    for i,s in enumerate(l):#type: str
        l[i] = s.replace('"',"").replace(" ","")
    return l
df_mot_cle = df_mot_cle.map(delete_quote)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
df_light = df_mot_cle[:10000]
mlb = MultiLabelBinarizer()

res = pd.DataFrame(mlb.fit_transform(df_light),columns=mlb.classes_,index=df_light.index)
res

In [ ]:
len(res)

In [ ]:
sum_df = res.sum().sort_values(ascending=False)
sum_df

In [ ]:
sum_df.apply(lambda x: x/len(res)*100)

In [ ]:
biggest_df = sum_df[:200]
biggest_df

In [ ]:
res_biggest = res[biggest_df.index]

In [ ]:
corr = res_biggest.corr()

In [ ]:
corr = corr.round(4)

In [ ]:
import seaborn  as sns
sns.heatmap(corr)

In [ ]:
dict_cor = corr.to_dict()
for el in dict_cor: #type:dict
    dict_cor[el] = {k: v for k,v in dict_cor[el].items() if ( dict_cor[el][k] > 0.5 or dict_cor[el][k] < -0.6)and el != k}

In [ ]:
dict_cor

In [ ]:
import graphviz as gz

def get_color(w: float):
    color=["green","blue","orange","red","black"]
    return color[int((w*len(color)))]

def dict_to_graph(d_in) -> gz.Graph:
	gra = gz.Graph(strict=True)
	for k, d in d_in.items():
		for k2, v in d.items():
			gra.edge(k, k2, label=str(v),color=get_color(v))
	return gra
g = dict_to_graph(dict_cor)

In [ ]:
g.render("test",format="png")

In [ ]:
g